# Step2 : 検索実行 (Question-Answering) - 回答例

ここでは、ユーザーからの質問に対し、Step 1 で作成したテキストとベクトル (Embedding) のデータ (```embeddings_data.csv```) の中から答えを探して回答する仕組みを構築していきましょう。

> ただし、Step 1 を完了していない方は、リポジトリ内の ```embeddings_data_backup.csv``` の名前を ```embeddings_data.csv``` に変更してください。

まず、準備として、OpenAI Python Library を Azure OpenAI Service に接続するための設定をおこないます。

ただし、**下記の embedding_model と qa_model の値は、事前に皆さんの環境にあわせて変更してください。**

In [1]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base =  os.environ["OPENAI_API_BASE"]
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]

embedding_model = "text-embedding-ada-002-deploy"
qa_model = "gpt-35-turbo-deploy"

Knowledge Base (```embeddings_data.csv```) から、全ドキュメントの Embedding 情報を読み込みます。<br>
列として、```text``` (本文), ```n_tokens``` (トークン数), ```embeddings``` (ベクトルのリスト) が含まれていることを確認してください。

In [2]:
import pandas as pd
import numpy as np

df=pd.read_csv("embeddings_data.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

df.head()

,text,n_tokens,embeddings
0,手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989...,249,"[-0.011639331467449665, -0.0028115962632000446..."
2,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,555,"[-0.005988525692373514, -0.020313821732997894,..."
6,1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。 時節柄軍事色が強ま...,862,"[-0.0008580254507251084, -0.009806005284190178..."
9,終戦後、学生である手塚は戦時中に描き溜めた長編の中から『幽霊男』(『メトロポリス』の原型)と...,499,"[-0.012157808057963848, -0.006739071570336819,..."
10,漫画家としてデビューする前の1945年頃、2代目桂春団治が地方での自主興行を行う際のポスター...,460,"[-0.00843341276049614, -0.015178769826889038, ..."


下記のヒントを参考にしながら、質問に渡すコンテキストとして、質問と近いテキストを 1800 トークンを超えない範囲で取得する処理を作成してください。

ヒント
- 質問を Embedding します (Embedding 方法は、Step 1 を参考にしてください)
- ベクトルどうしの距離を測る関数はさまざま存在します。例えば、```openai.embeddings_utils.distances_from_embeddings``` が使用できます。(この関数の詳細は [こちら](https://github.com/openai/openai-python/blob/main/openai/embeddings_utils.py) を参照してください。)
- 上述の通り、トークン数の情報は ```n_tokens``` 列に含まれています。

In [ ]:
!pip3 install scikit-learn scipy

In [3]:
from openai.embeddings_utils import distances_from_embeddings

def create_context(
    question, df, max_len=1800,
):
    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine=embedding_model)["data"][0]["embedding"]

    # Get the distances from the embeddings
    df["distances"] = distances_from_embeddings(q_embeddings, df["embeddings"].values, distance_metric="cosine")

    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values("distances", ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row["n_tokens"] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

In [4]:
# 動作確認
create_context(
    "木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の発電所は?",
    df=df,
)

'大同電力が発足した第一次世界大戦後から大正末期にかけて、日本では5つの電力会社が業界内で巨大化していた。 この5社がいわゆる「五大電力」で、他の4社は関東の東京電灯、中京と九州北部の東邦電力、関西の宇治川電気、同じ卸売り会社の日本電力を指す。 電気事業の大型化の中で、大同電力は木曽川において積極的な水力開発を手がけることにより規模を拡大していった。 木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の大井発電所は当時国内で最大の発電出力を擁した。\n\n###\n\n1917年(大正6年)に水利権許可を得ていた木曽川の「大桑第一水力」は、水路が長すぎることから実際の開発にあたっては2地点に分割することになり、下流側は「須原水力」となった。 同地点は須原発電所として大同発足直後ただちに着工され、1922年(大正11年)7月に竣工した。 須原発電所は長野県西筑摩郡大桑村に設置され、水路式発電所で出力は9,200kW。 エッシャーウイス製水車、ウェスティングハウス・エレクトリック製発電機各2台と日立製作所製変圧器を設置した。 竣工とともに大桑発電所まで77キロボルト(kV)送電線が完成、大桑発電所より名古屋市北区八龍町の六郷変電所へ至る既設送電線(同年6月全線竣工)と接続し、名古屋方面への送電を開始する。 さらに翌1923年(大正12年)12月、大阪送電幹線が完成すると須原変電所を通じ154kVに昇圧した上で大阪方面へも送電するようになった。\n\n###\n\n桃山発電所は「大桑第一水力」の一部を計画変更して開発された。 すなわち、大桑第一水力を分割して生じた2地点のうち上流側にあたる「駒ヶ根水力」を、途中に景勝地寝覚の床があることからさらに分割して生じた2地点のうちの一つである(もう一つの地点は寝覚発電所となった)。 1922年8月に着工、翌1923年12月に完成した。 西筑摩郡上松町にあり、エッシャーウイス製水車2台とウェスティングハウス製の発電機2台および変圧器を設置して2万3,100kWの出力で発電する設計であった。 桃山では、東京方面への送電を見越して、周波数を50・60ヘルツ双方に対応するよう水車・発電機が設計された。 発電所に先立って1923年11月、須原発電所までの77kV送電線が竣工した。 翌1924年(大正13年)1月には東筑

つぎに、上記で抽出した (1800 token を超えない) コンテキストを使って、下記のプロンプトを渡して completion を実行します。<br>
返ってきた答えが、最終的な回答です。

----------

```
以下のテキストを使って下記の質問に答えてください。もし答えがない場合には、「私は知らない」と答えてください。

コンテキスト: {context}

---

質問: {question}
答え:
```

In [5]:
def answer_question(
    df,
    model,
    question,
    max_len=1800,
    max_tokens=150,
    stop_sequence=None
):
    context = create_context(
        question,
        df,
        max_len=max_len,
    )
    # Create a completions using the question and context
    response = openai.ChatCompletion.create(
        engine=qa_model,
        temperature=0,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=stop_sequence,
        messages=[
            {"role": "system", "content": "OpenAI で学習された大規模言語モデルです。可能な限り日本語で答えてください。"},
            {"role": "user", "content": f"以下のテキストを使って下記の質問に答えてください。もし答えがない場合には、「私は知らない」と答えてください。\n\nコンテキスト: {context}\n\n---\n\n質問: {question}\n答え:"}
        ]
    )    
    return response["choices"][0]["message"]["content"].strip()

下記の質問で試してみてください。<br>
これらの質問は、標準の ChatGPT (```gpt-35-turbo```) では答えられない内容です。

1. 「木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の発電所は?」 (答え : 大井発電所)
2. 「1948年に、占領軍総司令部民間情報局が静岡に設立したのは?」 (答え : アメリカ文化センター)
3. 「今日は何曜日?」 (答え : 私は知らない)

In [6]:
answer_question(df, question="木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の発電所は?", model=qa_model)

'1924年に完成したダム式の発電所は大井発電所です。'

In [7]:
answer_question(df, question="1948年に、占領軍総司令部民間情報局が静岡に設立したのは?", model=qa_model)

'占領軍総司令部民間情報局が静岡に設立したのは、静岡CIE図書館（のちにアメリカ文化センターと改称）です。'

In [8]:
answer_question(df, question="今日は何曜日", model=qa_model)

'私は知りません。コンテキストには日付や曜日に関する情報がありません。'